In [ ]:
'''
URJC / GIA / Aprendizaje Automático 1 / Curso 23-24
alfredo.cuesta@urjc.es
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score, confusion_matrix

plt.rcParams['figure.figsize']=(4,4)
np.set_printoptions(precision=2)
pd.set_option("display.precision", 4)
seed = 1460

# Ejemplo

Utilizando el conjunto de datos **california_housing**, que está en la carpeta *sample_data* de todo Google Colab queremos estimar si los ingresos de esa zona (en mediana) serán superiores o inferiores a 3.5 con un SVM.

## Creación del sistema clasificador

1. Cargamos el conjunto de datos de entrenamiento

In [ ]:
#- cargar data set
folder_name = 'sample_data/'
file_name = 'california_housing_train.csv'
trainSet = pd.read_csv(folder_name+file_name)
target = ['median_income']
train_target = trainSet[target]
trainSet.drop(target, axis=1, inplace=True)
features = trainSet.columns

2. Obtenemos la etiqueta a partir de la columna "_median_income_"

In [ ]:
train_target = (train_target > 3.5)*1

3. Separamos el conjunto de entrenamiento en dos para poder evaluar el modelo
  - Entrenamiento = 80%
  - Validación = 20%

In [ ]:
val_size = 0.2

X_train, X_val, Y_train, Y_val = train_test_split(trainSet, train_target,
                                              stratify=train_target, shuffle=True,
                                              random_state=seed)

4. Con SVM conviene tener las características escaladas al intervalo $[0,1]$ o al $[-1,1]$.

  En este caso vamos a utilizar "_MaxAbsScaler_"

En este ejemplo NO vamos a hacer ningún otro preprocesado.

In [ ]:
scaler = MaxAbsScaler().set_output(transform='pandas')

scaler.fit(X_train)
X_train_scl = scaler.transform(X_train)

5. Entrenar un modelo SVM para clasificación; es decir SVC.

In [ ]:
max_iter = 2**12  #<- cambiando el exponente duplicamos
C_svc = 2**4      #<- lo mismo
kernel = 'linear' #<- linear, rbf, poly, ... ver documentación

svc_model = SVC(kernel=kernel, C = C_svc, max_iter=max_iter)
svc_model.fit(X_train_scl, Y_train.values.ravel())
#- SVM de Scikit-Learn no trabaja bien con Pandas.
#  Se le puede pasar Y_train pero devuelve mensajes de aviso ('warnings')
#  Por eso se extran los valores y se convierten en un array de un solo eje.

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=4096).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVC(C=16, kernel='linear', max_iter=4096)

6. Una vez aprendido el modelo, vemos que tal funciona con el conjunto de validación; que son ejemplos que NUNCA ha visto el modelo, igual que ocurrirá con el conjunto de test.

In [ ]:
X_val_scl = scaler.transform(X_val)
y_pred = svc_model.predict(X_val_scl)

# Calcular la precisión
accuracy = accuracy_score(Y_val, y_pred)
# Calcular la matriz de confusión
cf_mat = confusion_matrix(Y_val, y_pred)

print(f'accuracy = {accuracy:0.3f}\n')
print(f'Matriz de confusion = \n {pd.DataFrame(cf_mat)}')

accuracy = 0.848

Matriz de confusion = 
       0     1
0  1766   308
1   340  1836


7. Hemos obtenido un 84% de _accuracy_. No está mal :)

  Vamos a ver el modelo

In [ ]:
svc_model.__dict__

{'decision_function_shape': 'ovr',
 'break_ties': False,
 'kernel': 'linear',
 'degree': 3,
 'gamma': 'scale',
 'coef0': 0.0,
 'tol': 0.001,
 'C': 16,
 'nu': 0.0,
 'epsilon': 0.0,
 'shrinking': True,
 'probability': False,
 'cache_size': 200,
 'class_weight': None,
 'verbose': False,
 'max_iter': 4096,
 'random_state': None,
 '_sparse': False,
 'feature_names_in_': array(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
        'total_bedrooms', 'population', 'households', 'median_house_value'],
       dtype=object),
 'n_features_in_': 8,
 'class_weight_': array([1., 1.]),
 'classes_': array([0, 1]),
 '_gamma': 0.474578582545789,
 'support_': array([    2,     3,    20, ..., 12744, 12745, 12748], dtype=int32),
 'support_vectors_': array([[-0.94,  0.8 ,  0.25, ...,  0.04,  0.07,  0.18],
        [-0.98,  0.88,  0.4 , ...,  0.01,  0.02,  0.33],
        [-0.95,  0.81,  0.56, ...,  0.02,  0.07,  0.39],
        ...,
        [-0.98,  0.89,  0.31, ...,  0.05,  0.14,  0.4 ],
      

## Inferencia con el sistema clasificador

- ¿Qué pasaría si desplegamos este sistema en el destino? <br>
Le llegarán ejemplos nuevos y hará estimaciones de su etiqueta.

- ¿Qué tal funcionaría?<br>
Por "suerte" el conjunto de test tiene etiquetas (lo que no pasará cuando de verdad lo pongamos en funcionamiento).

Vamos a cargar el conjunto de test y ver qué tal funciona con nuestro sistema.

In [ ]:
#- cargar conjunto de test
folder_name = 'sample_data/'
file_name = 'california_housing_test.csv'
X_test = pd.read_csv(folder_name+file_name)
target = ['median_income']
Y_test = X_test[target]
X_test.drop(target, axis=1, inplace=True)
features = X_test.columns

#- convertir la etiqueta a 0 y 1
Y_test = (Y_test > 3.5)*1

#- alimentar los datos al sistema
X_test_scl = scaler.transform(X_test)
y_hat = svc_model.predict(X_test_scl)

#- Calcular la precisión
accuracy = accuracy_score(Y_test, y_hat)
#- Calcular la matriz de confusión
cf_mat = confusion_matrix(Y_test, y_hat)

print(f'accuracy = {accuracy:0.3f}\n')
print(f'Matriz de confusion = \n {pd.DataFrame(cf_mat)}')

accuracy = 0.841

Matriz de confusion = 
       0     1
0  1268   240
1   236  1256


# Ejercicios

1. leer la documentaciòn de SVC y modificar el paso 6 para probar otros modelos con otras opciones; por ejemplo otros kernel, diferentes hiperparámetros, etc.

2. Probar varios modelos y luego dibujar sus curvas ROC.

  ¿Qué modelo es mejor según el área bajo la ROC (AUROC) ?

3. Hacer una visualización del conjunto de entrenamiento y el clasificador utilizado. Para ello conviene utilizar PCA por lo que tendrás que repetir el código añadiendo PCA en el preprocesado.

  Recuerda que para crear la imagen hay que hacer un barrido del intervalo que vayas a mostrar. En este caso el plano será (pca0, pca1).